## 환경 초기 설정

In [ ]:
# User: Please enter your Project ID in this cell.
PROJECT_ID = 'your-gcp-project' # <-- ENTER YOUR ACTUAL PROJECT ID HERE!

# Verify that PROJECT_ID is not empty. If it is, raise an error.
if not PROJECT_ID:
    raise ValueError("ERROR: PROJECT_ID is not set. Please enter your Project ID above.")

print(f"Project ID set to: {PROJECT_ID}")

이제 이 셀을 실행하여 태스크 3 환경을 초기화합니다. 이 셀은 태스크 3에 필요한 라이브러리를 가져오고, BigQuery 클라이언트를 초기화하며, 사용할 주요 변수(테이블 ID 등)를 정의합니다.

In [ ]:
from google.cloud import bigquery
import pandas as pd
import pandas_gbq
from IPython.display import display

# Ensure PROJECT_ID has been defined in the cell above.
# You must run the 'Set Your Project ID' cell above before running this one.
if 'PROJECT_ID' not in locals() or not PROJECT_ID:
    raise ValueError("ERROR: PROJECT_ID is not set. Please run the 'Set Your Project ID' cell above first.")

client = bigquery.Client(project=PROJECT_ID, location="us-central1") # Explicitly pass project argument here

DATASET_ID = 'cymbal'
GEMINI_MODEL_NAME = f'{PROJECT_ID}.{DATASET_ID}.gemini_flash_model'

print(f"BigQuery Client Initialized for Project ID: {PROJECT_ID}")

def run_bq_query(sql: str, client: bigquery.Client):
    try:
        query_job = client.query(sql)
        print(f"Job {query_job.job_id} in state {query_job.state}")
        if query_job.statement_type == 'SELECT':
            df = query_job.to_dataframe()
            print(f"Query complete. Fetched {len(df)} rows.")
            return df
        else:
            query_job.result()
            print(f"Query for statement type {query_job.statement_type} complete.")
            return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

## 불만족 리뷰 고객 식별
### 이 쿼리는 'negative' 혹은 'neutral' 로 분류된 리뷰를 남긴 고객을 식별합니다.

In [ ]:
sql_get_negative = f"""

CREATE OR REPLACE TABLE `cymbal.negative_customers_list` AS
SELECT
    customer_id
FROM
    `cymbal.final_customer_insights`
WHERE
    ((JSON_EXTRACT_SCALAR(sentiment_json_string, '$.sentiment') = 'negative') OR (JSON_EXTRACT_SCALAR(sentiment_json_string, '$.sentiment') = 'neutral'))
GROUP BY customer_id;
"""
df_negative_customers = run_bq_query(sql_get_negative, client)
if df_negative_customers is not None:
    display(df_negative_customers)


## 불만족 고객 테이블 확인

In [ ]:
sql_show_table = "SELECT * FROM `cymbal.negative_customers_list` LIMIT 5;"

print("Fetching data from the new table...")
df_new_table_contents = run_bq_query(sql_show_table, client)

if df_new_table_contents is not None:
    display(df_new_table_contents)

## 불만족 고객의 세그먼트 및 지리적 데이터 검색
### 이 쿼리는 불만족 고객의 페르소나 및 지리적 정보를 수집합니다.

In [ ]:
sql_create_negative_segment_data = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_ID}.negative_customer_segment_data` AS
SELECT
    ncl.customer_id,
    t1.persona_age_group_profile,
    t2.address_city
FROM
    `{DATASET_ID}.negative_customers_list` AS ncl
JOIN
    `{DATASET_ID}.final_customer_insights` AS t1 ON ncl.customer_id = t1.customer_id
JOIN
    `{DATASET_ID}.customers` AS t2 ON ncl.customer_id = t2.customer_id;
"""

run_bq_query(sql_create_negative_segment_data, client)
print("✅ negative_customer_segment_data 테이블 생성 완료.")

## 불만족 고객 정보 테이블 확인

In [ ]:
sql_show_table = "SELECT * FROM `cymbal.negative_customer_segment_data` LIMIT 5;"

print("Fetching data from the new table...")
df_new_table_contents = run_bq_query(sql_show_table, client)

if df_new_table_contents is not None:
    display(df_new_table_contents)

## 세그먼트, 도시별 인기 제품 추출


### 세그먼트별, 도시별 고객들이 많이 구입한 상품을 수집하는 쿼리를 작성합니다.

#### 요구사항:
**segment_top_products_ranked 테이블**:
* 컬럼명: persona_age_group_profile, segment_top1_product, segment_top2_product
* final_customer_insights 테이블과 customer_reviews_external 테이블을 조인하여 사용합니다.
* ROW_NUMBER() 윈도우 함수를 사용하여 persona_age_group_profile별로 productId의 구매 횟수(COUNT(product_id))에 따라 순위를 매깁니다. 
* 상위 1위(rn=1)와 2위(rn=2) 제품의 product_id를 추출하여 각각 segment_top1_product, segment_top2_product 컬럼에 저장합니다. 

**city_top_products_ranked 테이블** :
* 컬럼명: address_city, city_top1_product, city_top2_product 
* customers 테이블과 customer_reviews_external 테이블을 조인하여 사용합니다. 
* ROW_NUMBER() 윈도우 함수를 사용하여 address_city별로 product_id의 구매 횟수(COUNT(product_id))에 따라 순위를 매깁니다. 
* 상위 1위(rn=1)와 2위(rn=2) 제품의 product_id를 추출하여 각각 city_top1_product, city_top2_product 컬럼에 저장합니다. 

힌트: 두 쿼리 모두 WITH 절을 사용하여 순위가 매겨진 중간 결과를 생성하고, 이후 MAX(CASE WHEN ... END) 패턴을 사용하여 피벗하여 최종 컬럼을 만들 수 있습니다.

In [ ]:
# 여기에 segment_top_products_ranked 테이블을 생성하는 SQL 쿼리를 작성하세요.
sql_create_segment_top_products = f"""
-- 이 부분에 SQL 쿼리를 직접 작성하여 cymbal.segment_top_products_ranked 테이블을 생성하세요.
-- 힌트: WITH 절과 ROW_NUMBER()를 사용하여 세그먼트별 상위 2개 제품을 찾고,
-- MAX(CASE WHEN ...)를 사용하여 컬럼으로 변환하세요.
"""

run_bq_query(sql_create_segment_top_products, client)
print("✅ segment_top_products_ranked 테이블 생성 완료.")

# 여기에 city_top_products_ranked 테이블을 생성하는 SQL 쿼리를 작성하세요.
sql_create_city_top_products = f"""
-- 이 부분에 SQL 쿼리를 직접 작성하여 cymbal.city_top_products_ranked 테이블을 생성하세요.
-- 힌트: WITH 절과 ROW_NUMBER()를 사용하여 도시별 상위 2개 제품을 찾고,
-- MAX(CASE WHEN ...)를 사용하여 컬럼으로 변환하세요.
"""

run_bq_query(sql_create_city_top_products, client)
print("✅ city_top_products_ranked 테이블 생성 완료.")

### 세그먼트별 인기 상품 테이블 확인

In [ ]:
sql_show_table = "SELECT * FROM `cymbal.segment_top_products_ranked` LIMIT 5;"

print("Fetching data from the new table...")
df_new_table_contents = run_bq_query(sql_show_table, client)

if df_new_table_contents is not None:
    display(df_new_table_contents)

### 도시별 인기 상품 테이블 확인

In [ ]:
sql_show_table = "SELECT * FROM `cymbal.city_top_products_ranked` LIMIT 5;"

print("Fetching data from the new table...")
df_new_table_contents = run_bq_query(sql_show_table, client)

if df_new_table_contents is not None:
    display(df_new_table_contents)

### 인기 상품 테이블 수합

위의 코드 생성을 완료한 후 아래 코드를 실행해 하나의 임시 테이블로 수합합니다.

In [ ]:
sql_create_candidate_products = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_ID}.temp_candidate_products_list` AS
SELECT
    ncl.customer_id,
    c.email,
    c.first_name,
    fci.persona_age_group_profile AS segment,
    ARRAY<STRING>[
        CAST(t_seg.segment_top1_product AS STRING),
        CAST(t_seg.segment_top2_product AS STRING),
        CAST(t_city.city_top1_product AS STRING),
        CAST(t_city.city_top2_product AS STRING)
    ] AS candidate_products
FROM
    `{DATASET_ID}.negative_customers_list` AS ncl
JOIN
    `{DATASET_ID}.final_customer_insights` AS fci
    ON ncl.customer_id = fci.customer_id
JOIN
    `{DATASET_ID}.customers` AS c
    ON ncl.customer_id = c.customer_id
LEFT JOIN
    `{DATASET_ID}.segment_top_products_ranked` AS t_seg
    ON fci.persona_age_group_profile = t_seg.persona_age_group_profile
LEFT JOIN
    `{DATASET_ID}.city_top_products_ranked` AS t_city
    ON c.address_city = t_city.address_city
WHERE
    fci.persona_age_group_profile IS NOT NULL
    AND c.address_city IS NOT NULL;
"""

run_bq_query(sql_create_candidate_products, client)
print(f"✅ 1단계 완료: `{PROJECT_ID}.{DATASET_ID}.temp_candidate_products_list` 생성.")

## 최종 추천 상품 테이블 생성
### 세그먼트, 지역별 추천을 고객이 리뷰한 상품과의 중복을 제외한 3가지 상품으로 최종 결정합니다.

고객의 지역 기반, 세그먼트 기반 추천 상품 4가지를 하나의 임시 테이블로 합칩니다.

In [ ]:
sql_create_candidate_products = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_ID}.temp_candidate_products_list` AS
SELECT
    ncl.customer_id,
    c.email,
    c.first_name,
    fci.persona_age_group_profile AS segment,
    ARRAY<STRING>[
        CAST(t_seg.segment_top1_product AS STRING),
        CAST(t_seg.segment_top2_product AS STRING),
        CAST(t_city.city_top1_product AS STRING),
        CAST(t_city.city_top2_product AS STRING)
    ] AS candidate_products -- 총 4개의 후보 상품 배열 (모두 STRING 타입으로 통일)
FROM
    `{DATASET_ID}.negative_customers_list` AS ncl
JOIN
    `{DATASET_ID}.final_customer_insights` AS fci
    ON ncl.customer_id = fci.customer_id
JOIN
    `{DATASET_ID}.customers` AS c
    ON ncl.customer_id = c.customer_id
LEFT JOIN
    `{DATASET_ID}.segment_top_products_ranked` AS t_seg
    ON fci.persona_age_group_profile = t_seg.persona_age_group_profile
LEFT JOIN
    `{DATASET_ID}.city_top_products_ranked` AS t_city
    ON c.address_city = t_city.address_city
WHERE
    fci.persona_age_group_profile IS NOT NULL
    AND c.address_city IS NOT NULL;
"""
run_bq_query(sql_create_candidate_products, client)
print(f"✅ `{PROJECT_ID}.{DATASET_ID}.temp_candidate_products_list` 생성 완료.")

임시 테이블에서 고객이 리뷰한 상품이 추천된 경우 추천에서 제외하고, 아닌 경우 도시 기반의 두 번째 추천 상품을 제외합니다.

In [ ]:
sql_create_final_recommendation_array = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_ID}.temp_final_recommendation_array` AS
WITH ExpandedCandidates AS (
    SELECT
        t1.customer_id,
        t1.email,
        t1.first_name,
        t1.segment,
        t_candidate_product AS candidate_product_item
    FROM
        `{DATASET_ID}.temp_candidate_products_list` AS t1,
        UNNEST(t1.candidate_products) AS t_candidate_product
),
PurchasedProductsExpanded AS (
    SELECT DISTINCT
        customer_id,
        CAST(product_id AS STRING) AS purchased_product_item
    FROM
        `{DATASET_ID}.customer_reviews_external`
    WHERE
        product_id IS NOT NULL
),
ExcludedProducts AS (
    SELECT
        ec.customer_id,
        ec.email,
        ec.first_name,
        ec.segment,
        ec.candidate_product_item,
        CASE
            WHEN ppe.purchased_product_item IS NULL THEN ec.candidate_product_item
            ELSE NULL
        END AS recommended_product_item_if_new
    FROM
        ExpandedCandidates AS ec
    LEFT JOIN
        PurchasedProductsExpanded AS ppe
        ON ec.customer_id = ppe.customer_id
        AND ec.candidate_product_item = ppe.purchased_product_item
    GROUP BY 1, 2, 3, 4, 5, ppe.purchased_product_item
),
FinalCandidates AS (
    SELECT
        ep.customer_id,
        ep.email,
        ep.first_name,
        ep.segment,
        ARRAY_AGG(recommended_product_item_if_new IGNORE NULLS ORDER BY
            -- 상품 선정 우선순위: Segment Top 1, Segment Top 2, City Top 1
            CASE
                WHEN recommended_product_item_if_new = CAST(t_city.city_top1_product AS STRING) THEN 3
                WHEN recommended_product_item_if_new = CAST(t_seg.segment_top2_product AS STRING) THEN 2
                WHEN recommended_product_item_if_new = CAST(t_seg.segment_top1_product AS STRING) THEN 1
                ELSE 4
            END
        ) AS final_recommended_products_with_exclusion
    FROM
        ExcludedProducts AS ep
    JOIN
        `{DATASET_ID}.segment_top_products_ranked` AS t_seg ON ep.segment = t_seg.persona_age_group_profile
    JOIN
        `{DATASET_ID}.customers` AS c ON ep.customer_id = c.customer_id
    JOIN
        `{DATASET_ID}.city_top_products_ranked` AS t_city ON c.address_city = t_city.address_city
    GROUP BY 1, 2, 3, 4
)
SELECT
    customer_id,
    email,
    segment,
    first_name,
    final_recommended_products_with_exclusion AS final_products_array
FROM
    FinalCandidates;
"""

run_bq_query(sql_create_final_recommendation_array, client)
print(f"✅ `{PROJECT_ID}.{DATASET_ID}.temp_final_recommendation_array` 생성 완료.")

## 최종 추천 상품 테이블 정리 및 저장

제공된 코드를 실행해 추천 상품 테이블을 저장합니다.

In [ ]:
sql_create_final_recommendations = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_ID}.final_personalized_recommendations` AS
SELECT
    customer_id,
    first_name as customer_name,
    email as customer_email,
    segment,
    final_products_array[OFFSET(0)] AS product1,
    final_products_array[OFFSET(1)] AS product2,
    final_products_array[OFFSET(2)] AS product3
FROM
    `{DATASET_ID}.temp_final_recommendation_array`;
"""

run_bq_query(sql_create_final_recommendations, client)
print(f"✅ `{PROJECT_ID}.{DATASET_ID}.final_personalized_recommendations` 테이블 생성 완료.")

### 추천 상품 테이블 확인

In [ ]:
print("\n--- 최종 추천 결과 테이블 미리보기 ---")
df_final_recs = run_bq_query(f"SELECT * FROM `{PROJECT_ID}.{DATASET_ID}.final_personalized_recommendations` LIMIT 5", client)
if df_final_recs is not None:
    display(df_final_recs)

## Gemini로 개인화된 추천 상품 평가

추천된 상품의 상품 이름과 카테고리, 고객의 세그먼트 정보를 불러와 하나의 테이블에 저장합니다.

In [ ]:
sql_create_product_details_table = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_ID}.temp_recommendation_details` AS
WITH ProductsExpanded AS (
    SELECT
        customer_id,
        email,
        first_name,
        segment,
        t_prod AS product_id,
        t_offset AS rank
    FROM
        `{DATASET_ID}.temp_final_recommendation_array`,
        UNNEST(final_products_array) AS t_prod WITH OFFSET AS t_offset
    WHERE
        t_prod IS NOT NULL AND t_offset < 3
)
SELECT
    pe.customer_id,
    pe.email,
    pe.first_name,
    pe.segment,
    pe.rank + 1 AS recommendation_rank,
    p.title AS product_title,
    p.categories AS product_categories,
    t_analysis.analysis AS gemini_persona_analysis
FROM
    ProductsExpanded AS pe
JOIN
    `{DATASET_ID}.products` AS p
    ON pe.product_id = CAST(p.product_id AS STRING)
JOIN
    `{DATASET_ID}.segment_level_gemini_analysis` AS t_analysis
    ON pe.segment = t_analysis.profile_name
ORDER BY
    pe.customer_id, recommendation_rank;
"""

run_bq_query(sql_create_product_details_table, client)
print(f"✅ `{PROJECT_ID}.{DATASET_ID}.temp_recommendation_details` 테이블 생성 완료")

### 평가 프롬프트 생성 및 저장

temp_recommendation_details 테이블에서 필요한 데이터를 로드하고, GEMINI_EVALUATION_PROMPT_TEMPLATE을 활용하여 각 추천 사항에 대한 Gemini 평가 프롬프트를 동적으로 생성합니다. \
생성된 프롬프트 DataFrame을 cymbal.temp_gemini_evaluation_prompts라는 임시 BigQuery 테이블에 저장하는 코드를 작성합니다.

In [ ]:
GEMINI_EVALUATION_PROMPT_TEMPLATE = """
당신은 고객 세그먼트 분석가입니다. 다음 정보를 기반으로, 추천 상품이 고객 세그먼트 페르소나에 얼마나 적합한지 평가하는 JSON 객체를 생성하세요.

1.  고객 세그먼트 페르소나 (분석 결과): {persona_analysis}
2.  추천 상품 이름: {product_title}
3.  추천 상품 카테고리: {product_categories}

JSON 형식 제약조건:
* "product_title" (상품 이름)
* "product_categories" (상품 카테고리)
* "compatibility_score" (페르소나 대비 적합성 점수, 1에서 100 사이의 정수)
* "reasoning" (점수를 부여한 근거, 50단어 이내)
* 출력은 JSON 객체 하나여야 합니다.
"""

TEMP_EVAL_PROMPT_TABLE_ID = "temp_gemini_evaluation_prompts"
TEMP_EVAL_TABLE = f"{PROJECT_ID}.{DATASET_ID}.{TEMP_EVAL_PROMPT_TABLE_ID}"

sql_get_eval_data = f"SELECT customer_id, recommendation_rank, product_title, product_categories, gemini_persona_analysis FROM `{DATASET_ID}.temp_recommendation_details`"
df_eval_data = run_bq_query(sql_get_eval_data, client)

if df_eval_data is not None and not df_eval_data.empty:
    # 여기에 prompts_df_eval DataFrame을 생성하고
    # cymbal.temp_gemini_evaluation_prompts 테이블에 저장하는 Python 코드를 직접 작성하세요.
    # 힌트: df_eval_data.apply(lambda row: ..., axis=1)와 pandas_gbq.to_gbq를 사용하세요.
    pass # 이 줄을 삭제하고 코드를 작성하세요.

else:
    print("⚠️ 평가 프롬프트를 생성할 상품 데이터가 없습니다.")

### Gemini 모델 호출 및 평가 테이블 생성

cymbal.temp_gemini_evaluation_prompts 테이블의 프롬프트를 사용해 BigQuery ML의 ML.GENERATE_TEXT 함수를 호출하고, Gemini의 평가 결과를 파싱하여 cymbal.gemini_recommendation_evaluation 테이블을 생성하는 SQL 쿼리를 작성합니다. \
JSON 결과 파싱 및 최종 테이블 생성 로직은 제공되니, ML.GENERATE_TEXT 호출 부분만 완성하세요.

* 모델 이름: GEMINI_MODEL_NAME 변수를 사용합니다.
* STRUCT 옵션: STRUCT(0.5 AS temperature, 1024 AS max_output_tokens, TRUE AS flatten_json_output)를 사용합니다.
* 힌트: PromptData CTE에서 임시 프롬프트 테이블({TEMP_EVAL_TABLE})을 선택하고, GeminiOutput CTE의 ML.GENERATE_TEXT 함수 호출 부분을 완성하세요.

In [ ]:
table_id_evaluation_output = f"{DATASET_ID}.gemini_recommendation_evaluation"

# 여기에 Gemini 모델을 호출하고 평가 테이블을 생성하는 SQL 쿼리를 작성하세요.
sql_batch_evaluation = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{table_id_evaluation_output}` AS
WITH PromptData AS (
    SELECT
        customer_id,
        recommendation_rank,
        product_title,
        prompt
    FROM
        `{TEMP_EVAL_TABLE}`
),
GeminiOutput AS (
    SELECT
        ml_generate_text_llm_result AS raw_json_output,
        prompt AS original_prompt_text
    FROM
        -- 이 부분에 ML.GENERATE_TEXT 함수를 호출하는 코드를 작성하세요.
        -- 모델 이름: `{GEMINI_MODEL_NAME}`
        -- 입력 데이터: (SELECT prompt FROM PromptData)
        -- STRUCT 옵션: STRUCT(0.5 AS temperature, 1024 AS max_output_tokens, TRUE AS flatten_json_output)
        -- 예시:
        -- ML.GENERATE_TEXT(
        --     MODEL `프로젝트_ID.데이터셋_ID.모델_이름`,
        --     (SELECT prompt FROM PromptData),
        --     STRUCT(0.5 AS temperature, 1024 AS max_output_tokens, TRUE AS flatten_json_output)
        -- )
        
        -- 이 아래에 ML.GENERATE_TEXT 호출 코드를 작성하세요 


),
JoinedOutput AS (
    SELECT
        t1.customer_id,
        t1.recommendation_rank,
        t1.product_title,
        t2.raw_json_output
    FROM
        PromptData AS t1
    JOIN
        GeminiOutput AS t2
        ON t1.prompt = t2.original_prompt_text
)
SELECT
    j.customer_id,
    j.recommendation_rank,
    j.product_title,
    j.raw_json_output AS gemini_raw_evaluation,
    CAST(JSON_EXTRACT_SCALAR(TRIM(REGEXP_REPLACE(j.raw_json_output, r'(?i)(^```json\\s*|\\s*```$)', '')), '$.compatibility_score') AS INT64) AS compatibility_score,
    TRIM(JSON_EXTRACT_SCALAR(TRIM(REGEXP_REPLACE(j.raw_json_output, r'(?i)(^```json\\s*|\\s*```$)', '')), '$.reasoning')) AS evaluation_reasoning
FROM
    JoinedOutput AS j
ORDER BY
    j.customer_id, j.recommendation_rank;
"""

run_bq_query(sql_batch_evaluation, client)
print(f"✅ 적합성 평가 테이블 `{PROJECT_ID}.{table_id_evaluation_output}` 생성 완료.")